In [ ]:
import htm

In [ ]:
from htm.bindings.sdr import SDR

from htm.algorithms import TemporalMemory as TM

import numpy as np

In [ ]:
from htm.algorithms import Connections

In [ ]:
def formatSdr(sdr):
    flatten = sdr.dense.any(axis=-1).astype(np.int8)
    result = ''
    for i in range(flatten.size):
        if i > 0 and i % 8 == 0:
            result += ' '
        result += str(flatten[i])
    return result

In [ ]:
bits = 1
n_values = 3
arraySize = bits*n_values

inputSDR = SDR(arraySize)

tm = TM(columnDimensions = (inputSDR.size, ),
        cellsPerColumn = 2,       # default: 32
        minThreshold = bits,         # default: 10
        activationThreshold = bits,  # default: 13
        initialPermanence = 0.5,
        maxNewSynapseCount = bits,
        maxSynapsesPerSegment = bits
       )

cycleArray = [0, 1, 2]

In [ ]:
for sensorValue in cycleArray:
    sensorValueBits = inputSDR.dense
    sensorValueBits = np.zeros(arraySize)
    sensorValueBits[sensorValue*bits:(sensorValue+1)*bits] = 1
    inputSDR.dense = sensorValueBits
    
    tm.compute(inputSDR, learn=True)
    print('V:'+ format(sensorValue,'>2') + ' |', formatSdr(tm.getActiveCells()), 'Active')
    
    tm.activateDendrites(True)
    print(format(tm.anomaly, '.2f') + ' |', formatSdr(tm.getPredictiveCells()), 'Predicted')
    
    for segment in range(tm.connections.numSegments()):
        print(f'segment: {segment} synapses: {tm.connections.synapsesForSegment(segment)}')
        for synapse in tm.connections.synapsesForSegment(segment):
            print(f'synapse: {synapse} -> cell: {tm.connections.presynapticCellForSynapse(synapse)}')

In [ ]:
sensorValue=0
sensorValueBits = inputSDR.dense
sensorValueBits = np.zeros(arraySize)
sensorValueBits[sensorValue*bits:(sensorValue+1)*bits] = 1
inputSDR.dense = sensorValueBits
tm.compute(inputSDR, learn=True)

In [ ]:
dir(tm.connections)

In [ ]:
connections = tm.connections

In [ ]:
dir(tm)

In [ ]:
connections.bumpSegment?

In [ ]:
tm.connections.numSegments()

In [ ]:
tm.getActiveCells().sparse

In [ ]:
tm.getActiveSegments()

In [ ]:
tm.getMatchingSegments()

In [ ]:
tm.getPredictiveCells().sparse

In [ ]:
tm.getWinnerCells().sparse

In [ ]:
dir(tm.connections)

In [ ]:
tm.connections.idxOnCellForSegment(50)

In [ ]:
tm.connections.permanenceForSynapse(1)

In [ ]:
tm.connections.segmentsForCell(6)

In [ ]:
for segment in range(tm.connections.numSegments()):
    print(tm.connections.synapsesForSegment(segment))

In [ ]:
tm.connections.presynapticCellForSynapse(16)

In [ ]:
tm.getActiveCells?

In [ ]:
tm.connections.numSegments()

In [ ]:
def get_prediction_forward(steps, initial_value, tm):
    arraySize = 80
    inputSDR = SDR(arraySize)
    sensorValue = initial_value
    sensorValueBits = inputSDR.dense
    sensorValueBits = np.zeros(arraySize)
    sensorValueBits[sensorValue*8:(sensorValue+1)*8] = 1
    inputSDR.dense = sensorValueBits
    for k in range(steps):
        tm.compute(inputSDR, learn=False)
        tm.activateDendrites(False)
        inputSDR.dense = tm.getPredictiveCells().dense.any(axis=-1).astype(np.int8)
        print('step:'+ format(k+1,'>2') + ' |', formatSdr(tm.getPredictiveCells()))

In [ ]:
get_prediction_forward(10, 2, tm)

In [ ]:
a = tm.getActiveCells().dense

In [ ]:
a

In [ ]:
a.any(axis=-1).astype(np.int8)[80]